In [ ]:
import dspy
import anaconda_ai.integrations.litellm  # noqa: F401

In [ ]:
lm = dspy.LM(
    "anaconda/TinyLlama-1.1B-Chat-v1.0_Q4_K_M.gguf",
    cache=False,
    temperature=0.1,
    optional_params={"server": {"ctx_size": 512}},
)
dspy.configure(lm=lm)

In [ ]:
res = lm("what is pi?")
print(res[0])

In [ ]:
fact_checking = dspy.asyncify(dspy.ChainOfThought("claims -> verdicts: list[bool]"))

res = await fact_checking(
    claims=["Python was released in 1991.", "Python is a compiled language."]
)
res

In [ ]:
embedder = dspy.Embedder("anaconda/bge-small-en-v1.5/q4_k_m", batch_size=100)
embeddings = embedder(["hello", "world"])

In [ ]:
embeddings.shape

In [ ]:
output = await embedder.acall(["cat"])
output.shape